In [1]:
# Here we will study in detail about applying aggregate and transformation functions in more detail to each group

In [2]:
# We will perform different operations like split, calculate standard deviation, count and mean

In [3]:
# Apply within a group normalization, linear regression, rank etc

In [4]:
# pivot tables and perform quantile analysis and other stastical group analyses

In [5]:
import numpy as np

import pandas as pd

import os as o

import sys as ss

import csv as cv

import json as jayz

#import lxml as xml Not working ecen though it is installed


import requests

import sqlite3

# import sqlalchemy as sqa , not working even though installed


import re

import matplotlib.pyplot as plt

from datetime import datetime

import seaborn as sns

# Now if you want to do the plotting in pycharm, remember that no matter which method you use the figure output will
# not come unless until you write plt.show()


from io import StringIO


## How to think about Grouping Operations

In [6]:
# The order to be followed here is that we start with first splitting the data object then apply function and finally combining

# This can be either pandas Series or DataFrame. So for ex in DataFrame we can group the data on either axis i.e. on rows or columns

# Then we start with the splitting the data on the basis of correspondence to keys that can be a list or dict

# Finally we combine the data obtained from applying the functions

In [7]:
sd = pd.DataFrame({'kiss1':['a','a',None,'b','b','a',None],
                  'kiss2':pd.Series([1,2,1,2,1,None,1],dtype="Float64"),
                   'data1':np.random.standard_normal((7)),
                  'data2':np.random.standard_normal((7))
                  })

sd

,kiss1,kiss2,data1,data2
0,a,1.0,0.620174,0.179602
1,a,2.0,0.883183,0.002364
2,None,1.0,1.142955,-0.575137
3,b,2.0,-0.994904,1.078923
4,b,1.0,0.430778,0.318213
5,a,<NA>,-0.957275,0.806904
6,None,1.0,-1.115764,-1.293537


In [8]:
# Now suppose we want to find the mean of data1 column using the labels of kiss1. There are a number of ways to do this,
# but here we will do it using the groupby function.

In [9]:
grouper = sd['data1'].groupby(sd['kiss1'])
grouper

In [10]:
# Now, here the grouper variable is an object of group by function. Currently it has not performed any computation or calculation
# It only has some information required to perform all the operations

In [11]:
grouper.mean()

kiss1
a    0.182027
b   -0.282063
Name: data1, dtype: float64

In [12]:
# Now, here we can perform group by using multiple columns as well.

In [13]:
multiplegrouper =  sd['data1'].groupby([sd['kiss1'],sd['kiss2']])

multiplegrouper

In [14]:
multiplegrouper.mean()

kiss1  kiss2
a      1.0      0.620174
       2.0      0.883183
b      1.0      0.430778
       2.0     -0.994904
Name: data1, dtype: float64

In [15]:
# Now, here we grouped the data using two keys and hence the resultant series has heiarchial indexing.

In [16]:
# multiplegrouper.unstack()

In [17]:
# Now we can apply a different object like list for aggregation

In [18]:
sd

,kiss1,kiss2,data1,data2
0,a,1.0,0.620174,0.179602
1,a,2.0,0.883183,0.002364
2,None,1.0,1.142955,-0.575137
3,b,2.0,-0.994904,1.078923
4,b,1.0,0.430778,0.318213
5,a,<NA>,-0.957275,0.806904
6,None,1.0,-1.115764,-1.293537


In [19]:
states = ['MH','BI','BI','MH','MH','BI','MH']

saal = [2005,2005,2006,2005,2006,2005,2006]

len(states) == len(saal)

True

In [20]:
sd['data1'].groupby([saal,states]).mean()

2005  BI   -0.037046
      MH   -0.187365
2006  BI    1.142955
      MH   -0.342493
Name: data1, dtype: float64

In [21]:
sd['data1'].groupby([states,saal]).mean()

BI  2005   -0.037046
    2006    1.142955
MH  2005   -0.187365
    2006   -0.342493
Name: data1, dtype: float64

In [22]:
# Now, we can use the above mentioned functions in a more quicker way

In [23]:
sd.groupby('kiss1').mean() # However, the result will be a little different as the remaining columns will be included
 # If in groupby the column passed is the first one and it is non numeric

,kiss2,data1,data2
kiss1,,,
a,1.5,0.182027,0.329623
b,1.5,-0.282063,0.698568


In [24]:
sd.groupby('kiss2').mean() # Now, kiss1 column has been discarded since it is non-numeric

,data1,data2
kiss2,,
1.0,0.269536,-0.342715
2.0,-0.055860,0.540644


In [25]:
sd.groupby([sd['kiss1'],sd['kiss2']]).mean()

data1     data2
kiss1 kiss2                    
a     1.0    0.620174  0.179602
      2.0    0.883183  0.002364
b     1.0    0.430778  0.318213
      2.0   -0.994904  1.078923

In [26]:
sd.groupby('kiss2') # This acts like the grouper variable used before

In [27]:
# Now, an useful method of the groupby is the size method

In [28]:
sd.groupby([sd['kiss1'],sd['kiss2']]).size()

kiss1  kiss2
a      1.0      1
       2.0      1
b      1.0      1
       2.0      1
dtype: int64

In [29]:
# By default all null and non-numerical values are excluded from the result of size. But you can include the null by
# disabling the dropna function

In [30]:
sd.groupby([sd['kiss1'],sd['kiss2']],dropna=False).size()

kiss1  kiss2
a      1.0      1
       2.0      1
       NaN      1
b      1.0      1
       2.0      1
NaN    1.0      2
dtype: int64

In [31]:
# A similar function in spirit to size is the count function that computes the number of non null values in each group

In [32]:
sd.groupby([sd['kiss1']]).count()

,kiss2,data1,data2
kiss1,,,
a,2,3,3
b,2,2,2


### Iterating over Groups

In [33]:
# Now, the object returned by groupby function supports iteration, generating a sequence of 2 tuples containing the group name
# and a chunk of data

In [34]:
for name,group in sd.groupby('kiss1'):
    
    print(name,' name hai')
    print(group,'  group hai')

a  name hai
  kiss1  kiss2     data1     data2
0     a    1.0  0.620174  0.179602
1     a    2.0  0.883183  0.002364
5     a   <NA> -0.957275  0.806904   group hai
b  name hai
  kiss1  kiss2     data1     data2
3     b    2.0 -0.994904  1.078923
4     b    1.0  0.430778  0.318213   group hai


In [35]:
# Now, in case of multiple keys, the first element will be tuple of key values

In [36]:
for (k1,k2),group in sd.groupby(['kiss1','kiss2']):
    print((k1,k2))
    print(group)

('a', 1.0)
  kiss1  kiss2     data1     data2
0     a    1.0  0.620174  0.179602
('a', 2.0)
  kiss1  kiss2     data1     data2
1     a    2.0  0.883183  0.002364
('b', 1.0)
  kiss1  kiss2     data1     data2
4     b    1.0  0.430778  0.318213
('b', 2.0)
  kiss1  kiss2     data1     data2
3     b    2.0 -0.994904  1.078923


In [37]:
# Now you can do many things with the data, but one useful method is converting them to dict

In [38]:
crisis = {name:fame for name,fame in sd.groupby('kiss1')}

crisis

{'a':   kiss1  kiss2     data1     data2
 0     a    1.0  0.620174  0.179602
 1     a    2.0  0.883183  0.002364
 5     a   <NA> -0.957275  0.806904,
 'b':   kiss1  kiss2     data1     data2
 3     b    2.0 -0.994904  1.078923
 4     b    1.0  0.430778  0.318213}

In [39]:
sd

,kiss1,kiss2,data1,data2
0,a,1.0,0.620174,0.179602
1,a,2.0,0.883183,0.002364
2,None,1.0,1.142955,-0.575137
3,b,2.0,-0.994904,1.078923
4,b,1.0,0.430778,0.318213
5,a,<NA>,-0.957275,0.806904
6,None,1.0,-1.115764,-1.293537


In [40]:
# Now, we can also represent the column names of our sd DataFrame using dictionary and depict them as values

In [41]:
grouping = sd.groupby({'kiss1':'key','kiss2':'key','data1':'data','data2':'data'
                       
                      },axis='columns')

grouping

# Now, note that you cannot apply normal functions here as there is no numeric value column wise

In [42]:
for group_key,group_value in grouping:
    print(group_key,'  key hai \n')
    print(group_value,'  value haii')

data   key hai 

      data1     data2
0  0.620174  0.179602
1  0.883183  0.002364
2  1.142955 -0.575137
3 -0.994904  1.078923
4  0.430778  0.318213
5 -0.957275  0.806904
6 -1.115764 -1.293537   value haii
key   key hai 

  kiss1  kiss2
0     a    1.0
1     a    2.0
2  None    1.0
3     b    2.0
4     b    1.0
5     a   <NA>
6  None    1.0   value haii


### selecting a column or a subset of a column

In [43]:
# Now, suppose you want to give an index to a group by result obtained. So you do that in the following way

In [44]:
sd

,kiss1,kiss2,data1,data2
0,a,1.0,0.620174,0.179602
1,a,2.0,0.883183,0.002364
2,None,1.0,1.142955,-0.575137
3,b,2.0,-0.994904,1.078923
4,b,1.0,0.430778,0.318213
5,a,<NA>,-0.957275,0.806904
6,None,1.0,-1.115764,-1.293537


In [45]:
sd.groupby('kiss1')['data1']

sd.groupby('kiss1')['data1'].mean()

kiss1
a    0.182027
b   -0.282063
Name: data1, dtype: float64

In [46]:

# the above example can be written in the following way as well and it works just same finee with all the functions and methods

sd['data1'].groupby(sd['kiss1'])

sd['data1'].groupby(sd['kiss1']).mean()


# Abhi zara group by ko dhyan se samjho. The order that you see in group by sd['data1'].groupby(sd['kiss1']) is not how 

# the groupby is working. Actually how you read it and how it works is that humlog kiss1 ke basis pe data1 ko group kar rah
# hai. Simply said humlog kiss1 ke basis pe data1 ko divide kar rahe hai, toh iss liye jiske basis pe grouping hoti hai
# yaani jo data ko divide kar raha hai vahi groupby mei jaayega na.concatenated_names_data = pd.concat(mylist)


kiss1
a    0.182027
b   -0.282063
Name: data1, dtype: float64

In [47]:
# Now, one other important thing is that suppose you wanted to compute mean of just one column, you can do that in

In [48]:
sd.groupby([sd['kiss1'],sd['kiss2']])[['data1']].mean()

data1
kiss1 kiss2          
a     1.0    0.620174
      2.0    0.883183
b     1.0    0.430778
      2.0   -0.994904

### Grouping with Dictionaries and Series

In [49]:
peeps = pd.DataFrame(np.random.standard_normal((5,5)),
                    columns=['a','b','c','d','e' ],
                    index = ['monty','bunty','shonty','rocky','jacky'])

peeps

,a,b,c,d,e
monty,-1.382438,-0.454494,0.149370,-0.852749,0.248126
bunty,-1.909013,-0.274240,0.400298,-0.289848,0.064038
shonty,-0.242472,0.141680,-0.067206,-0.814898,2.386655
rocky,-0.788254,0.402752,1.485252,-1.264374,-0.332434
jacky,-1.047905,-1.123365,-0.007143,-0.546681,0.243186


In [50]:
# Now, suppose you want to groupby using a  different object like series or dictionary we can do that as follows

In [51]:
mapps = {'a':'red','b':'red','c':'blue','d':'blue','e':'red','z':'blood red'}

In [52]:
peeps.iloc[2:3,[1,2]] = np.nan

peeps

,a,b,c,d,e
monty,-1.382438,-0.454494,0.149370,-0.852749,0.248126
bunty,-1.909013,-0.274240,0.400298,-0.289848,0.064038
shonty,-0.242472,NaN,NaN,-0.814898,2.386655
rocky,-0.788254,0.402752,1.485252,-1.264374,-0.332434
jacky,-1.047905,-1.123365,-0.007143,-0.546681,0.243186


In [53]:
mappsed = peeps.groupby(mapps,axis='columns')

In [54]:
mappsed.sum()

,blue,red
monty,-0.703379,-1.588806
bunty,0.110450,-2.119216
shonty,-0.814898,2.144183
rocky,0.220878,-0.717936
jacky,-0.553824,-1.928084


In [55]:
# We can do the similar thing with series as follows

In [56]:
mappsed_series = pd.Series(mapps)

mappsed_series

a          red
b          red
c         blue
d         blue
e          red
z    blood red
dtype: object

In [57]:
mappsed_series_grouper = peeps.groupby(mappsed_series,axis='columns')

In [58]:
mappsed_series_grouper.sum()

,blue,red
monty,-0.703379,-1.588806
bunty,0.110450,-2.119216
shonty,-0.814898,2.144183
rocky,0.220878,-0.717936
jacky,-0.553824,-1.928084


In [59]:
mappsed_series_grouper.count()

,blue,red
monty,2,3
bunty,2,3
shonty,1,2
rocky,2,3
jacky,2,3


### Grouping with functions

In [60]:
peeps

,a,b,c,d,e
monty,-1.382438,-0.454494,0.149370,-0.852749,0.248126
bunty,-1.909013,-0.274240,0.400298,-0.289848,0.064038
shonty,-0.242472,NaN,NaN,-0.814898,2.386655
rocky,-0.788254,0.402752,1.485252,-1.264374,-0.332434
jacky,-1.047905,-1.123365,-0.007143,-0.546681,0.243186


In [61]:
# Now, you can use different functions to be used for grouping. The function will be called index or column wise as specified

In [62]:
peeps.groupby(len,axis='columns').sum()

,1
monty,-2.292185
bunty,-2.008766
shonty,1.329284
rocky,-0.497058
jacky,-2.481908


In [63]:
peeps.groupby(len).sum()

,a,b,c,d,e
5,-5.127610,-1.449347,2.027777,-2.953651,0.222915
6,-0.242472,0.000000,0.000000,-0.814898,2.386655


### Grouping by index Levels

In [64]:
# Now, for DataFrames that have multiple indexing levels for them we can use grouping in following ways

In [65]:
colsmulti = pd.MultiIndex.from_arrays([['MU','MU','MU','DU','DU'],
                                     [1,3,5,1,3]],
                                     names=['city','universities'])

colsmulti

MultiIndex([('MU', 1),
            ('MU', 3),
            ('MU', 5),
            ('DU', 1),
            ('DU', 3)],
           names=['city', 'universities'])

In [66]:
exampmultindgrping = pd.DataFrame(np.random.standard_normal((4,5)),
                                 columns=colsmulti
                                 )

exampmultindgrping

city                MU                            DU          
universities         1         3         5         1         3
0             0.856884 -0.807857 -0.230492 -0.809235 -0.184937
1             0.561458 -1.263343  0.888620 -0.042525  0.247886
2            -0.044066  1.439323 -0.427996  0.835674  0.174667
3            -0.180465 -0.474830  0.620847 -1.116341 -0.974777

In [67]:
# Now, to apply grouping pass the index level number or name in the levels attribute

In [68]:
exampmultindgrping.groupby(level='city',axis='columns').mean()

city,DU,MU
0,-0.497086,-0.060488
1,0.102680,0.062245
2,0.505170,0.322421
3,-1.045559,-0.011482


## Data Aggregation

In [69]:
# Aggregation refers to any form of data transformation that produces scalar values from arrays
# These methods include mean, max, min etc

In [70]:
sd

,kiss1,kiss2,data1,data2
0,a,1.0,0.620174,0.179602
1,a,2.0,0.883183,0.002364
2,None,1.0,1.142955,-0.575137
3,b,2.0,-0.994904,1.078923
4,b,1.0,0.430778,0.318213
5,a,<NA>,-0.957275,0.806904
6,None,1.0,-1.115764,-1.293537


In [71]:
# Now we will learn how to call different functions with groupby, they might not give the most optimized results but they help
# in understanding the data better

In [72]:
grps = sd.groupby('kiss1')

In [73]:
grps.mean()

,kiss2,data1,data2
kiss1,,,
a,1.5,0.182027,0.329623
b,1.5,-0.282063,0.698568


In [74]:
print(grps['data1'].nsmallest(1))

grps['data1'].nlargest(1)

# Here, for example if you see the nlargest output first column in output is the column on which groupby has been applied
# After that we have the index value in second column and then lastly we have the largest value for the data1 col.
# Here, the function has been applied in respect with the kiss1 and data1 cols

kiss1   
a      5   -0.957275
b      3   -0.994904
Name: data1, dtype: float64


kiss1   
a      1    0.883183
b      4    0.430778
Name: data1, dtype: float64

In [75]:
sd

,kiss1,kiss2,data1,data2
0,a,1.0,0.620174,0.179602
1,a,2.0,0.883183,0.002364
2,None,1.0,1.142955,-0.575137
3,b,2.0,-0.994904,1.078923
4,b,1.0,0.430778,0.318213
5,a,<NA>,-0.957275,0.806904
6,None,1.0,-1.115764,-1.293537


In [76]:
# Now, you can pass your own aggregate functions to a groupby object by using agg method

In [77]:
def peke_to_peke(arr):
    return arr.max() - arr.min()

In [78]:
grps.agg(peke_to_peke)

,kiss2,data1,data2
kiss1,,,
a,1.0,1.840458,0.804539
b,1.0,1.425682,0.760710


In [79]:
# Some other methods like describe also work even though they are not aggregate

In [80]:
grps.describe()

kiss2                                           data1            ...  \
      count mean       std  min   25%  50%   75%  max count      mean  ...   
kiss1                                                                  ...   
a       2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0  0.182027  ...   
b       2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0 -0.282063  ...   

                          data2                                          \
            75%       max count      mean       std       min       25%   
kiss1                                                                     
a      0.751679  0.883183   3.0  0.329623  0.422730  0.002364  0.090983   
b      0.074358  0.430778   2.0  0.698568  0.537903  0.318213  0.508391   

                                     
            50%       75%       max  
kiss1                                
a      0.179602  0.493253  0.806904  
b      0.698568  0.888746  1.078923  

[2 rows x 24 columns]

### Column-wise and Multiple Function Application

In [81]:
tipper = pd.read_csv('examples/tips.csv')

tipper

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4
...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3
240,27.18,2.00,Yes,Sat,Dinner,2
241,22.67,2.00,Yes,Sat,Dinner,2
242,17.82,1.75,No,Sat,Dinner,2


In [82]:
tipper['tip_prcnt'] = tipper['tip'] / tipper['total_bill']

tipper

,total_bill,tip,smoker,day,time,size,tip_prcnt
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [83]:
grouppups = tipper.groupby([tipper['day'],tipper['smoker']])

grouppups

In [84]:
grouppups.mean()

total_bill       tip      size  tip_prcnt
day  smoker                                           
Fri  No       18.420000  2.812500  2.250000   0.151650
     Yes      16.813333  2.714000  2.066667   0.174783
Sat  No       19.661778  3.102889  2.555556   0.158048
     Yes      21.276667  2.875476  2.476190   0.147906
Sun  No       20.506667  3.167895  2.929825   0.160113
     Yes      24.120000  3.516842  2.578947   0.187250
Thur No       17.113111  2.673778  2.488889   0.160298
     Yes      19.190588  3.030000  2.352941   0.163863

In [85]:
print(grouppups['smoker'].count())

grouppups['tip_prcnt'].mean()

day   smoker
Fri   No         4
      Yes       15
Sat   No        45
      Yes       42
Sun   No        57
      Yes       19
Thur  No        45
      Yes       17
Name: smoker, dtype: int64


day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_prcnt, dtype: float64

In [86]:
grouppups_tipprcnt = grouppups['tip_prcnt']

In [87]:
grouppups_tipprcnt.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_prcnt, dtype: float64

In [88]:
# Now, suppose you want to apply multiple functions. You can do so by passing them in a list and the
# column names will be the function names.

In [89]:
grouppups_tipprcnt.agg(['mean','std','median',peke_to_peke])

# Now, one more thing to notice here is that we have passed the mean method's name in quotes where as the function defined
# by us peke_to_peke is passed normally, that's because mean, std are pre-defined function and peke_to_peke is user-defined


mean       std    median  peke_to_peke
day  smoker                                            
Fri  No      0.151650  0.028123  0.149241      0.067349
     Yes     0.174783  0.051293  0.173913      0.159925
Sat  No      0.158048  0.039767  0.150152      0.235193
     Yes     0.147906  0.061375  0.153624      0.290095
Sun  No      0.160113  0.042347  0.161665      0.193226
     Yes     0.187250  0.154134  0.138122      0.644685
Thur No      0.160298  0.038774  0.153492      0.193350
     Yes     0.163863  0.039389  0.153846      0.151240

In [90]:
# Now, if you don't like the function names as column names you can change them in the following way

In [91]:
grouppups_tipprcnt.agg([('averagechandu','mean'),('ye apna standard hai',np.std)])

averagechandu  ye apna standard hai
day  smoker                                     
Fri  No           0.151650              0.028123
     Yes          0.174783              0.051293
Sat  No           0.158048              0.039767
     Yes          0.147906              0.061375
Sun  No           0.160113              0.042347
     Yes          0.187250              0.154134
Thur No           0.160298              0.038774
     Yes          0.163863              0.039389

In [92]:
# Now, here take a break and understand a few things
# 1) you started with grouppups = tipper.groupby([tipper['day'],tipper['smoker']]) that means you want to apply aggragate and
# grouping functions on the remaining columns with respect to these functions
# Then we studied how to apply  groupby and multiple aggregate functions with just one column
# grouppups_tipprcnt = grouppups['tip_prcnt'] and grouppups_tipprcnt.agg(['mean','std','median',peke_to_peke])

In [93]:
# Now we will study how to apply multiple aggregate functions with grouping on more than one column

In [94]:
funncs = ['mean','count','max','min']

ressults = grouppups['tip_prcnt','total_bill'].agg(funncs)

ressults

c:\python 3.7\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


tip_prcnt                           total_bill                    
                 mean count       max       min       mean count    max    min
day  smoker                                                                   
Fri  No      0.151650     4  0.187735  0.120385  18.420000     4  22.75  12.46
     Yes     0.174783    15  0.263480  0.103555  16.813333    15  40.17   5.75
Sat  No      0.158048    45  0.291990  0.056797  19.661778    45  48.33   7.25
     Yes     0.147906    42  0.325733  0.035638  21.276667    42  50.81   3.07
Sun  No      0.160113    57  0.252672  0.059447  20.506667    57  48.17   8.77
     Yes     0.187250    19  0.710345  0.065660  24.120000    19  45.35   7.25
Thur No      0.160298    45  0.266312  0.072961  17.113111    45  41.19   7.51
     Yes     0.163863    17  0.241255  0.090014  19.190588    17  43.11  10.34

In [95]:
ressults['total_bill']

mean  count    max    min
day  smoker                                
Fri  No      18.420000      4  22.75  12.46
     Yes     16.813333     15  40.17   5.75
Sat  No      19.661778     45  48.33   7.25
     Yes     21.276667     42  50.81   3.07
Sun  No      20.506667     57  48.17   8.77
     Yes     24.120000     19  45.35   7.25
Thur No      17.113111     45  41.19   7.51
     Yes     19.190588     17  43.11  10.34

In [96]:
# Just as before you can change the name of function and so on

In [97]:
func_and_names = [('apna average','mean'),('ginti hai','count'),('sabse zyada','max'),('nahi ke barobar','min')]

In [98]:
ressults_redefined = grouppups['tip_prcnt','total_bill'].agg(func_and_names)

ressults_redefined

c:\python 3.7\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


tip_prcnt                                         total_bill  \
            apna average ginti hai sabse zyada nahi ke barobar apna average   
day  smoker                                                                   
Fri  No         0.151650         4    0.187735        0.120385    18.420000   
     Yes        0.174783        15    0.263480        0.103555    16.813333   
Sat  No         0.158048        45    0.291990        0.056797    19.661778   
     Yes        0.147906        42    0.325733        0.035638    21.276667   
Sun  No         0.160113        57    0.252672        0.059447    20.506667   
     Yes        0.187250        19    0.710345        0.065660    24.120000   
Thur No         0.160298        45    0.266312        0.072961    17.113111   
     Yes        0.163863        17    0.241255        0.090014    19.190588   

                                                   
            ginti hai sabse zyada nahi ke barobar  
day  smoker                                        
Fri  No             4       22.75           12.46  
     Yes           15       40.17            5.75  
Sat  No            45       48.33            7.25  
     Yes           42       50.81            3.07  
Sun  No            57       48.17            8.77  
     Yes           19       45.35            7.25  
Thur No            45       41.19            7.51  
     Yes           17       43.11           10.34

In [99]:
# Now one more better method is there to do the same thing but with dictionaries

In [100]:
grouppups.agg({'tip_prcnt':[('chotu','min'),('sabse zyada','max'),('ginti ke chokre','count'),('standard apna',np.std)],
               'size':[('total vajan','sum'),('beech ka','median')]})

tip_prcnt                                                  size  \
                chotu sabse zyada ginti ke chokre standard apna total vajan   
day  smoker                                                                   
Fri  No      0.120385    0.187735               4      0.028123           9   
     Yes     0.103555    0.263480              15      0.051293          31   
Sat  No      0.056797    0.291990              45      0.039767         115   
     Yes     0.035638    0.325733              42      0.061375         104   
Sun  No      0.059447    0.252672              57      0.042347         167   
     Yes     0.065660    0.710345              19      0.154134          49   
Thur No      0.072961    0.266312              45      0.038774         112   
     Yes     0.090014    0.241255              17      0.039389          40   

                      
            beech ka  
day  smoker           
Fri  No            2  
     Yes           2  
Sat  No            2  
     Yes           2  
Sun  No            3  
     Yes           2  
Thur No            2  
     Yes           2

In [101]:
tipper

,total_bill,tip,smoker,day,time,size,tip_prcnt
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


## Apply: General split-apply-combine

In [102]:
# In this section we will be learning about the apply function. Here let's understand the working of apply.
# Apply first splits the object being manipulated, then attempts to apply the function on each piece. Finally it will
# concatenate the pieces together to give the required output

In [103]:
tipper

,total_bill,tip,smoker,day,time,size,tip_prcnt
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [104]:
# Now suppose from thr above DataFrame, you wanted to select the top 5 tip percent for the group. So do as follows

In [105]:
def top_tipprcnt(dataframer,n=5,column = 'tip_prcnt'):
    return dataframer.sort_values(column,ascending = False)[:n]
        

In [106]:
hs = [1233,134,59483,37477472,134429,1,843]

hs.sort()

hs[:4]

[1, 134, 843, 1233]

In [107]:
top_tipprcnt(tipper,n=6)

,total_bill,tip,smoker,day,time,size,tip_prcnt
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [108]:
# Now, we did this without using the apply function. So let's see the output after apply

In [109]:
tipper.groupby('smoker').apply(top_tipprcnt,n=6)

total_bill   tip smoker   day    time  size  tip_prcnt
smoker                                                            
No     232       11.61  3.39     No   Sat  Dinner     2   0.291990
       149        7.51  2.00     No  Thur   Lunch     2   0.266312
       51        10.29  2.60     No   Sun  Dinner     2   0.252672
       185       20.69  5.00     No   Sun  Dinner     5   0.241663
       88        24.71  5.85     No  Thur   Lunch     2   0.236746
       6          8.77  2.00     No   Sun  Dinner     2   0.228050
Yes    172        7.25  5.15    Yes   Sun  Dinner     2   0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2   0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1   0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4   0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2   0.279525
       93        16.32  4.30    Yes   Fri  Dinner     2   0.263480

In [110]:
# now, let's understand what happened here. First the data is split on basis of smoker, then we apply the data manipulation 
# techniques. Finally the data is returned and hierarchial index that contains index from the original DataFrame

In [111]:
# Now, suppose you want to pass other parameters of the function as well. So we do that in the following way

In [112]:
tipper.groupby(['smoker','day']).apply(top_tipprcnt,n=1,column = 'tip_prcnt')

total_bill   tip smoker   day    time  size  tip_prcnt
smoker day                                                             
No     Fri  223       15.98  3.00     No   Fri   Lunch     3   0.187735
       Sat  232       11.61  3.39     No   Sat  Dinner     2   0.291990
       Sun  51        10.29  2.60     No   Sun  Dinner     2   0.252672
       Thur 149        7.51  2.00     No  Thur   Lunch     2   0.266312
Yes    Fri  93        16.32  4.30    Yes   Fri  Dinner     2   0.263480
       Sat  67         3.07  1.00    Yes   Sat  Dinner     1   0.325733
       Sun  172        7.25  5.15    Yes   Sun  Dinner     2   0.710345
       Thur 194       16.58  4.00    Yes  Thur   Lunch     2   0.241255

In [113]:
tipper.groupby('smoker')['tip_prcnt'].mean()

smoker
No     0.159328
Yes    0.163196
Name: tip_prcnt, dtype: float64

In [114]:
tipper.groupby('smoker').apply(top_tipprcnt,n=6)
# Now, in this example you can see that the smoker part is coming twice which makes it kind of redundant.
#  So to solve this do as shown in the following block

total_bill   tip smoker   day    time  size  tip_prcnt
smoker                                                            
No     232       11.61  3.39     No   Sat  Dinner     2   0.291990
       149        7.51  2.00     No  Thur   Lunch     2   0.266312
       51        10.29  2.60     No   Sun  Dinner     2   0.252672
       185       20.69  5.00     No   Sun  Dinner     5   0.241663
       88        24.71  5.85     No  Thur   Lunch     2   0.236746
       6          8.77  2.00     No   Sun  Dinner     2   0.228050
Yes    172        7.25  5.15    Yes   Sun  Dinner     2   0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2   0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1   0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4   0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2   0.279525
       93        16.32  4.30    Yes   Fri  Dinner     2   0.263480

In [115]:
tipper.groupby('smoker',group_keys=False).apply(top_tipprcnt,n=6)


,total_bill,tip,smoker,day,time,size,tip_prcnt
232,11.61,3.39,No,Sat,Dinner,2,0.291990
149,7.51,2.00,No,Thur,Lunch,2,0.266312
51,10.29,2.60,No,Sun,Dinner,2,0.252672
185,20.69,5.00,No,Sun,Dinner,5,0.241663
88,24.71,5.85,No,Thur,Lunch,2,0.236746
6,8.77,2.00,No,Sun,Dinner,2,0.228050
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535


### Quantile and Bucket analysis

#### This topic is a little like, you get the point right. It is a little hard to understand soo....... just try to breeze through it

In [116]:
# Now here we will be using pandas.cut which is used to divide data into specific intervals. Then it can directly be passed
# to groupby function.

In [117]:
frames = pd.DataFrame({'datta1':np.random.standard_normal(1999),
                      'datta2':np.random.standard_normal(1999)})
frames

,datta1,datta2
0,1.714729,-0.216434
1,-1.431376,1.066907
2,-1.318887,-1.419106
3,0.485626,1.164799
4,0.086936,-1.709987
...,...,...
1994,-0.836538,0.606658
1995,0.530572,0.039879
1996,1.842022,-0.189785
1997,-0.534228,1.118336


In [118]:
cutpiece = pd.cut(frames['datta1'],4)

cutpiece

0          (1.439, 2.948]
1       (-1.577, -0.0686]
2       (-1.577, -0.0686]
3        (-0.0686, 1.439]
4        (-0.0686, 1.439]
              ...        
1994    (-1.577, -0.0686]
1995     (-0.0686, 1.439]
1996       (1.439, 2.948]
1997    (-1.577, -0.0686]
1998     (-0.0686, 1.439]
Name: datta1, Length: 1999, dtype: category
Categories (4, interval[float64]): [(-3.091, -1.577] < (-1.577, -0.0686] < (-0.0686, 1.439] < (1.439, 2.948]]

In [119]:
# Now, this can be directly passed to groupby, how it works is a little meh to understand

In [120]:
def get_stats(grps):
    return pd.DataFrame({'min':grps.min(),
                        'max':grps.max(),
                        'count':grps.count(),
                        'meane':grps.mean()})

In [121]:
grouper_nahi_samjha_mujhe = frames.groupby(cutpiece)

In [122]:
grouper_nahi_samjha_mujhe.apply(get_stats)

min       max  count     meane
datta1                                                       
(-3.091, -1.577]  datta1 -3.084756 -1.576750    106 -1.985651
                  datta2 -3.057113  2.266538    106 -0.078345
(-1.577, -0.0686] datta1 -1.576370 -0.068951    871 -0.685156
                  datta2 -3.152353  3.407919    871 -0.002522
(-0.0686, 1.439]  datta1 -0.064150  1.435625    882  0.552327
                  datta2 -3.675805  3.110455    882 -0.039096
(1.439, 2.948]    datta1  1.442387  2.947571    140  1.859261
                  datta2 -2.495536  2.577623    140  0.138899

In [123]:
# Now from observing what I have understood is that firt the cutpiece was created by dividing only the datta1 column of frames
# Then, we used groupby on frames and grouped it using the intervals of cutpiece that had datta1 column in intervals
# and finally applied the get_stats on the entire frames DataFrame and grouped it using the cutpiece interval only

In [124]:
# Instead of using the function, we can do it in the following way as well

In [125]:
grouper_nahi_samjha_mujhe.agg(['min','max','count','mean'])

datta1                              datta2            \
                        min       max count      mean       min       max   
datta1                                                                      
(-3.091, -1.577]  -3.084756 -1.576750   106 -1.985651 -3.057113  2.266538   
(-1.577, -0.0686] -1.576370 -0.068951   871 -0.685156 -3.152353  3.407919   
(-0.0686, 1.439]  -0.064150  1.435625   882  0.552327 -3.675805  3.110455   
(1.439, 2.948]     1.442387  2.947571   140  1.859261 -2.495536  2.577623   

                                   
                  count      mean  
datta1                             
(-3.091, -1.577]    106 -0.078345  
(-1.577, -0.0686]   871 -0.002522  
(-0.0686, 1.439]    882 -0.039096  
(1.439, 2.948]      140  0.138899

In [126]:
# Now, we will see examples using qcut

In [127]:
cutpiece_qcut = pd.qcut(frames['datta1'],4,labels=False)

cutpiece_qcut

0       3
1       0
2       0
3       2
4       2
       ..
1994    0
1995    2
1996    3
1997    1
1998    3
Name: datta1, Length: 1999, dtype: int64

In [128]:
cutpiece_qcut = pd.qcut(frames['datta1'],4)

cutpiece_qcut

0          (0.617, 2.948]
1        (-3.086, -0.703]
2        (-3.086, -0.703]
3        (-0.0447, 0.617]
4        (-0.0447, 0.617]
              ...        
1994     (-3.086, -0.703]
1995     (-0.0447, 0.617]
1996       (0.617, 2.948]
1997    (-0.703, -0.0447]
1998       (0.617, 2.948]
Name: datta1, Length: 1999, dtype: category
Categories (4, interval[float64]): [(-3.086, -0.703] < (-0.703, -0.0447] < (-0.0447, 0.617] < (0.617, 2.948]]

In [129]:
# Now, the differnce that we get from passing labels = False is that with labels = True you get intervals, and
# with labels = False you get the indices, indices is nothing but the last rounded of value of the respective interval

In [130]:
# Now, applying the functions

In [131]:
grouper_nahi_samjha_mujhe_with_qcut = frames.groupby(cutpiece_qcut)


In [132]:

grouper_nahi_samjha_mujhe_with_qcut.apply(get_stats)

min       max  count     meane
datta1                                                       
(-3.086, -0.703]  datta1 -3.084756 -0.704280    500 -1.263418
                  datta2 -3.152353  2.357974    500 -0.013235
(-0.703, -0.0447] datta1 -0.701257 -0.044661    500 -0.353561
                  datta2 -2.771574  3.407919    500 -0.018402
(-0.0447, 0.617]  datta1 -0.043984  0.616783    499  0.273550
                  datta2 -3.675805  3.110455    499 -0.027466
(0.617, 2.948]    datta1  0.617806  2.947571    500  1.224377
                  datta2 -2.916969  2.577623    500  0.007973

### Filling missing values with group specific values

In [133]:
# Now, if our DataFrame has missing values, then we can always drop them. But one other approach is to fill it using
# Values like mean or max

In [134]:
sos = pd.Series(np.random.standard_normal(31))

sos

0    -0.042966
1    -1.167990
2    -0.772755
3     2.698432
4     0.311617
5     1.106388
6     0.966056
7     0.730479
8    -0.087687
9     0.175859
10   -0.685351
11   -0.717839
12    1.764608
13    0.810216
14   -0.470116
15    0.251954
16   -1.387328
17   -1.238828
18   -0.653853
19   -0.858804
20    0.929638
21    0.985304
22    1.872896
23   -0.160069
24    0.117660
25   -0.302303
26   -0.169646
27   -0.906516
28    0.404772
29   -0.835368
30   -1.605129
dtype: float64

In [135]:
sos[::2] = np.nan

sos

0          NaN
1    -1.167990
2          NaN
3     2.698432
4          NaN
5     1.106388
6          NaN
7     0.730479
8          NaN
9     0.175859
10         NaN
11   -0.717839
12         NaN
13    0.810216
14         NaN
15    0.251954
16         NaN
17   -1.238828
18         NaN
19   -0.858804
20         NaN
21    0.985304
22         NaN
23   -0.160069
24         NaN
25   -0.302303
26         NaN
27   -0.906516
28         NaN
29   -0.835368
30         NaN
dtype: float64

In [136]:
sos.fillna(sos.mean())

0     0.038061
1    -1.167990
2     0.038061
3     2.698432
4     0.038061
5     1.106388
6     0.038061
7     0.730479
8     0.038061
9     0.175859
10    0.038061
11   -0.717839
12    0.038061
13    0.810216
14    0.038061
15    0.251954
16    0.038061
17   -1.238828
18    0.038061
19   -0.858804
20    0.038061
21    0.985304
22    0.038061
23   -0.160069
24    0.038061
25   -0.302303
26    0.038061
27   -0.906516
28    0.038061
29   -0.835368
30    0.038061
dtype: float64

In [137]:
cities = ['Amsterdam','Leiden','Hague','Gronnigen','Rotterdam','Utretcht','Erasmus','Nijmegen']



group_cities = ['East','East','East','East',
                'West','West','West','West']

len(cities) == len(group_cities)

True

In [138]:
data_cities = pd.Series(np.random.standard_normal(len(cities)),index=cities)

data_cities

Amsterdam   -0.178350
Leiden       1.886427
Hague       -2.386356
Gronnigen   -0.144804
Rotterdam   -0.927125
Utretcht     0.767057
Erasmus     -0.330897
Nijmegen     0.807434
dtype: float64

In [139]:
# Now, let's add some missing values

In [140]:
data_cities[['Hague','Utretcht','Nijmegen']] = np.nan

data_cities

Amsterdam   -0.178350
Leiden       1.886427
Hague             NaN
Gronnigen   -0.144804
Rotterdam   -0.927125
Utretcht          NaN
Erasmus     -0.330897
Nijmegen          NaN
dtype: float64

In [141]:
data_cities.groupby(group_cities).mean()

East    0.521091
West   -0.629011
dtype: float64

In [142]:
data_cities.groupby(group_cities).count()

East    3
West    2
dtype: int64

In [143]:
# Now, to fill the missing values

In [144]:
def fill_them_missers(data_incomplete):
    
    return data_incomplete.fillna(data_incomplete.mean())

In [145]:
data_cities.groupby(group_cities).apply(fill_them_missers)

Amsterdam   -0.178350
Leiden       1.886427
Hague        0.521091
Gronnigen   -0.144804
Rotterdam   -0.927125
Utretcht    -0.629011
Erasmus     -0.330897
Nijmegen    -0.629011
dtype: float64

In [146]:
data_cities.fillna(data_cities.mean()) # or you can directly just do this

Amsterdam   -0.178350
Leiden       1.886427
Hague        0.061050
Gronnigen   -0.144804
Rotterdam   -0.927125
Utretcht     0.061050
Erasmus     -0.330897
Nijmegen     0.061050
dtype: float64

In [147]:
# But if you need specific values, then do this

In [148]:
fillers = {'East':-123,'West':-321}

In [149]:
def fill_them_missers_specific(data_incomplete):
    
    return data_incomplete.fillna(fillers[data_incomplete.name])  # Here a slight new introduction is the name attribute it's most likely calling
# the group values names. Not that understood it basically is used to call the dictionary values

In [150]:
data_cities.groupby(group_cities).apply(fill_them_missers_specific)

Amsterdam     -0.178350
Leiden         1.886427
Hague       -123.000000
Gronnigen     -0.144804
Rotterdam     -0.927125
Utretcht    -321.000000
Erasmus       -0.330897
Nijmegen    -321.000000
dtype: float64

### Example: Random Sampling and Permutation

In [151]:
# Now this is an interesting topic that teaches how to use Grouping for random sampling purposes. Here, we will see how
# if we have to draw out random cards then grouping can be used

In [152]:
suit_card = ['H','S','C','D'] # Hearts, spade,club, Diamonds

In [153]:
card_val = (list(range(1,11)) + [10] * 3 ) * 4

card_val 

# So here what has happened is that first we have created a list starting from 1 till 10. Then we have added
# a secondary list [10] * 3 which is nothing but [10,10,10]. Finally we have made the entire series repeat by 4 times

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 10,
 10,
 10,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 10,
 10,
 10,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 10,
 10,
 10,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 10,
 10,
 10]

In [154]:
bassi_names = ['A'] + list(range(2,11)) + ['J','K','Q']

bassi_names

['A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'K', 'Q']

In [155]:
emp_card =[]

In [156]:
for sweet in suit_card:
    emp_card.extend(sweet + str(baasi) for baasi in bassi_names)

In [157]:
emp_card

['HA',
 'H2',
 'H3',
 'H4',
 'H5',
 'H6',
 'H7',
 'H8',
 'H9',
 'H10',
 'HJ',
 'HK',
 'HQ',
 'SA',
 'S2',
 'S3',
 'S4',
 'S5',
 'S6',
 'S7',
 'S8',
 'S9',
 'S10',
 'SJ',
 'SK',
 'SQ',
 'CA',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'CJ',
 'CK',
 'CQ',
 'DA',
 'D2',
 'D3',
 'D4',
 'D5',
 'D6',
 'D7',
 'D8',
 'D9',
 'D10',
 'DJ',
 'DK',
 'DQ']

In [158]:
card_val

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 10,
 10,
 10,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 10,
 10,
 10,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 10,
 10,
 10,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 10,
 10,
 10]

In [159]:
emp_card

['HA',
 'H2',
 'H3',
 'H4',
 'H5',
 'H6',
 'H7',
 'H8',
 'H9',
 'H10',
 'HJ',
 'HK',
 'HQ',
 'SA',
 'S2',
 'S3',
 'S4',
 'S5',
 'S6',
 'S7',
 'S8',
 'S9',
 'S10',
 'SJ',
 'SK',
 'SQ',
 'CA',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'CJ',
 'CK',
 'CQ',
 'DA',
 'D2',
 'D3',
 'D4',
 'D5',
 'D6',
 'D7',
 'D8',
 'D9',
 'D10',
 'DJ',
 'DK',
 'DQ']

In [160]:
deck = pd.Series(card_val,index=emp_card)

deck

HA      1
H2      2
H3      3
H4      4
H5      5
H6      6
H7      7
H8      8
H9      9
H10    10
HJ     10
HK     10
HQ     10
SA      1
S2      2
S3      3
S4      4
S5      5
S6      6
S7      7
S8      8
S9      9
S10    10
SJ     10
SK     10
SQ     10
CA      1
C2      2
C3      3
C4      4
C5      5
C6      6
C7      7
C8      8
C9      9
C10    10
CJ     10
CK     10
CQ     10
DA      1
D2      2
D3      3
D4      4
D5      5
D6      6
D7      7
D8      8
D9      9
D10    10
DJ     10
DK     10
DQ     10
dtype: int64

In [161]:
# Now, suppose you want to withdraw 5 sample cards

In [162]:
def drawer(funcdef , n = 5):
    return funcdef.sample(n)

In [163]:
drawer(deck)

H2      2
C10    10
S8      8
S6      6
S3      3
dtype: int64

In [164]:
# Now, suppose you wanted to grp the cards individually, ex D7 as D and 7 separately so for that do this

In [165]:
def divider(deckf):
    # print(deckf[-1]) 
    return deckf[-1]  # Now what this will do is act as a slicing tool, i.e if you pass D7 it will give 7

In [166]:

deck.groupby(divider).apply(drawer,n = 2) # Now, here n = 2 part is a littleee kind of confusing like what I can understand is that
# First the divider gets called and it returns the last part of the card, and the last part is the important(it is the suit card) , as the entire
# set is made from combo of that only. Now we group the data based on those important last part of the card.
# Then finally since the length of the card is 2, hence for sampling we pass on the basis of the length only

0  D10    10
   S10    10
2  H2      2
   D2      2
3  S3      3
   C3      3
4  S4      4
   H4      4
5  H5      5
   D5      5
6  H6      6
   D6      6
7  H7      7
   D7      7
8  C8      8
   D8      8
9  C9      9
   D9      9
A  CA      1
   HA      1
J  CJ     10
   SJ     10
K  HK     10
   DK     10
Q  CQ     10
   SQ     10
dtype: int64

### Example: Group Weighted Average and Correlation

In [167]:
# In this section we will learn about weighted average between two series or DataFrame

In [168]:
dff = pd.DataFrame({'category':['a','a','a','a','b','b','b','b'],
                    'data':np.random.standard_normal(8),
                    'weights':np.random.uniform(size = 8)
                   })

dff

,category,data,weights
0,a,-1.735475,0.621478
1,a,-0.030035,0.162823
2,a,-0.080203,0.406010
3,a,0.543330,0.895063
4,b,0.522642,0.103160
5,b,0.228473,0.692738
6,b,-0.430899,0.895607
7,b,1.457111,0.436634


In [169]:
grouped_weights = dff.groupby('category')



In [170]:

grouped_weights.mean()

,data,weights
category,,
a,-0.325596,0.521343
b,0.444332,0.532035


In [171]:
# Now, to calculate weighted average

In [172]:
def get_weighavg(grouped_weights_func):
    
    return np.average(grouped_weights_func['data'],weights=grouped_weights_func['weights'])

# THen again did'nt understand this part that well as to what is being passed in the weight attribute and all

In [173]:
grouped_weights.apply(get_weighavg)

category
a   -0.301959
b    0.217324
dtype: float64

In [174]:
# Now, let's see another database that has the end of day or closing prices of few stocks

In [175]:
close_pristock = pd.read_csv('examples/stock_px.csv',parse_dates=True,index_col=0)

close_pristock

,AAPL,MSFT,XOM,SPX
2003-01-02,7.40,21.11,29.22,909.03
2003-01-03,7.45,21.14,29.24,908.59
2003-01-06,7.45,21.52,29.96,929.01
2003-01-07,7.43,21.93,28.95,922.93
2003-01-08,7.28,21.31,28.83,909.93
...,...,...,...,...
2011-10-10,388.81,26.94,76.28,1194.89
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66


In [176]:
close_pristock.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [177]:
# Now, this part is just on a roll.... could not understand it that well

In [178]:
# First is we calculate correlation of daily returns of the stock with the dataframe column

# We will do this with the help of a function

In [179]:
def spx_corr(group_spx):
    return group_spx.corrwith(group_spx['SPX'])

In [180]:
# Next we will compute the percent change on the dataframe using the prc_change() method

In [181]:
ressults = close_pristock.pct_change().dropna()

ressults

,AAPL,MSFT,XOM,SPX
2003-01-03,0.006757,0.001421,0.000684,-0.000484
2003-01-06,0.000000,0.017975,0.024624,0.022474
2003-01-07,-0.002685,0.019052,-0.033712,-0.006545
2003-01-08,-0.020188,-0.028272,-0.004145,-0.014086
2003-01-09,0.008242,0.029094,0.021159,0.019386
...,...,...,...,...
2011-10-10,0.051406,0.026286,0.036977,0.034125
2011-10-11,0.029526,0.002227,-0.000131,0.000544
2011-10-12,0.004747,-0.001481,0.011669,0.009795
2011-10-13,0.015515,0.008160,-0.010238,-0.002974


In [182]:
# Finally we group these percent changes over the years using a function that returns the year attribute of the datetime field

In [183]:
def get_year(yeardf):
    print(yeardf.year)
    return yeardf.year

In [184]:
by_year = ressults.groupby(get_year)

by_year

Int64Index([2003, 2003, 2003, 2003, 2003, 2003, 2003, 2003, 2003, 2003,
            ...
            2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011],
           dtype='int64', length=2213)


In [185]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [186]:
# Now, see the below example easy to understand

In [187]:
def corr_app_mic(funcg):
    return funcg['AAPL'].corr(funcg['MSFT'])

In [188]:
by_year.apply(corr_app_mic)

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### Example : Group-Wise Linear Regression

In [189]:
pip install statsmodels

Note: you may need to restart the kernel to use updated packages.


'c:\python' is not recognized as an internal or external command,
operable program or batch file.


In [190]:
# To see the code and explaination see pycharm GroupWiseLinearRegression.py notebook

# Then again, a little out of bounds for me and hard to understand

## Group Transforms and "Unwrapped" GroupBys

In [191]:
# In this section we will learn about the Transform function.
# It has more constraints on the kind of function you cab use

In [192]:
# The various characterstics of the function are

# It can produce a scalar value to be broadcast to the shape of the group
# It can produce an object of the same shape as the input group
# It must not mutate it's input

In [193]:
# Let's see an example

In [194]:
df = pd.DataFrame({'key':['a','b','v']*4,
                   'value':np.arange(12.)
                  })

df

,key,value
0,a,0.0
1,b,1.0
2,v,2.0
3,a,3.0
4,b,4.0
5,v,5.0
6,a,6.0
7,b,7.0
8,v,8.0
9,a,9.0


In [195]:
transform_topG = df['value'].groupby(df['key'])

transform_topG




In [196]:
transform_topG1 = df.groupby('key')['value']
transform_topG1 

# The example given in the book is of this block but I have written the entire section using both for better understanding


In [197]:
transform_topG == transform_topG1

False

In [198]:
df['value'].groupby(df['key']) == df.groupby('key')['value']

False

In [199]:
df['value'].groupby(df['key']).mean()

key
a    4.5
b    5.5
v    6.5
Name: value, dtype: float64

In [200]:
df.groupby('key')['value'].mean()

key
a    4.5
b    5.5
v    6.5
Name: value, dtype: float64

In [201]:
df.groupby('key')['value'].mean() == df['value'].groupby(df['key']).mean()

key
a    True
b    True
v    True
Name: value, dtype: bool

In [202]:
# Now, suppose we wanted to create a series that was of the same size as df['value'], but the series had the grouped average values
# Meaning that in place of the value for the key 'a' we will get it's average value of 4.5

In [203]:
def transformers(grptrnsf):
    print(grptrnsf)
    return grptrnsf.mean()

In [204]:






transform_topG.transform(transformers)



0    0.0
3    3.0
6    6.0
9    9.0
Name: a, dtype: float64
1      1.0
4      4.0
7      7.0
10    10.0
Name: b, dtype: float64
2      2.0
5      5.0
8      8.0
11    11.0
Name: v, dtype: float64


0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [205]:
transform_topG1.mean()

key
a    4.5
b    5.5
v    6.5
Name: value, dtype: float64

In [206]:


transform_topG1.transform(transformers)


0    0.0
3    3.0
6    6.0
9    9.0
Name: a, dtype: float64
1      1.0
4      4.0
7      7.0
10    10.0
Name: b, dtype: float64
2      2.0
5      5.0
8      8.0
11    11.0
Name: v, dtype: float64


0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [207]:
# For built in aggregate functions you can also use

In [208]:
transform_topG.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [209]:
# Like apply Transform also works with series and all, but the code must be written in such a way that result size 
# should be same as input size

In [210]:
def two_timer(grps):
    return grps * 2

In [211]:
transform_topG.transform(two_timer)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [212]:
# Next we will see how to calculate rank. Rank is a way of defining the importance of the value by some calculation.
# Check the internet for more

In [213]:
def ranker(funcgrp):
    return funcgrp.rank(ascending = False)



In [214]:
transform_topG.transform(ranker)

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [215]:
df

,key,value
0,a,0.0
1,b,1.0
2,v,2.0
3,a,3.0
4,b,4.0
5,v,5.0
6,a,6.0
7,b,7.0
8,v,8.0
9,a,9.0


In [216]:
# Now, let's see how transform and apply provide same values

In [217]:
def trans_and_apply_checker(dfunc):
    return (dfunc - dfunc.mean()) / dfunc.std()

In [218]:
transform_topG.transform(trans_and_apply_checker)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [219]:
transform_topG.apply(trans_and_apply_checker)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [220]:
transform_topG1.transform(trans_and_apply_checker)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [221]:
transform_topG1.apply(trans_and_apply_checker)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [222]:
# A little word of advise that the built-in aggregate functions such as mean and sum are a lot faster than the
# general apply function as they have a "fast path"

In [223]:
transform_topG.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [224]:
Normalize_is = (df['value'] - transform_topG.transform('mean')) / transform_topG.transform('std')

Normalize_is

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

## Pivot tables and Cross-Tabulations

In [225]:
# Now, pivot tables is a data summarization tool that is found in many spreadsheets and other data analysis software

# It aggregates a table of data by one or more keys and arranges the data in rectangle with some group keys along the rows and

# some along the column.

# In python Pivot table has been made possible the groupby function as seen in this section along with the very awesome
# pandas.pivot_table method. Then, we also have the DataFrame.pivot_table method for more specialization

In [226]:
# We will understand the pivot method by using the tipping database.

In [227]:
trippy_tip = pd.read_csv('examples/tips.csv')

trippy_tip

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4
...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3
240,27.18,2.00,Yes,Sat,Dinner,2
241,22.67,2.00,Yes,Sat,Dinner,2
242,17.82,1.75,No,Sat,Dinner,2


In [228]:
tipper

,total_bill,tip,smoker,day,time,size,tip_prcnt
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [229]:
tipper.pivot_table(index=['day','smoker'])

size       tip  tip_prcnt  total_bill
day  smoker                                           
Fri  No      2.250000  2.812500   0.151650   18.420000
     Yes     2.066667  2.714000   0.174783   16.813333
Sat  No      2.555556  3.102889   0.158048   19.661778
     Yes     2.476190  2.875476   0.147906   21.276667
Sun  No      2.929825  3.167895   0.160113   20.506667
     Yes     2.578947  3.516842   0.187250   24.120000
Thur No      2.488889  2.673778   0.160298   17.113111
     Yes     2.352941  3.030000   0.163863   19.190588

In [230]:
# Now, we very much know that the same result could have been obtained by

tipper.groupby(['day','smoker']).mean()

total_bill       tip      size  tip_prcnt
day  smoker                                           
Fri  No       18.420000  2.812500  2.250000   0.151650
     Yes      16.813333  2.714000  2.066667   0.174783
Sat  No       19.661778  3.102889  2.555556   0.158048
     Yes      21.276667  2.875476  2.476190   0.147906
Sun  No       20.506667  3.167895  2.929825   0.160113
     Yes      24.120000  3.516842  2.578947   0.187250
Thur No       17.113111  2.673778  2.488889   0.160298
     Yes      19.190588  3.030000  2.352941   0.163863

In [231]:
# Now, if you want to do more manipulation like average of only tip and size and then you want to do this average on basis of
# smokers. Also you want to have the day and time in the row column so for that we do

In [232]:
tipper.pivot_table(index=['day','time'],columns='smoker',
                 values = ['size','tip'] )

size                 tip          
smoker             No       Yes        No       Yes
day  time                                          
Fri  Dinner  2.000000  2.222222  2.750000  3.003333
     Lunch   3.000000  1.833333  3.000000  2.280000
Sat  Dinner  2.555556  2.476190  3.102889  2.875476
Sun  Dinner  2.929825  2.578947  3.167895  3.516842
Thur Dinner  2.000000       NaN  3.000000       NaN
     Lunch   2.500000  2.352941  2.666364  3.030000

In [233]:
# Now, here we will see about calculating the partial values which come in the all parameters. The all has the effect of adding 
# all the rows and columns or the ones that are being grouped in the column, like in our case we have the size and tip columns
# so for them the all function will have the mean of size's grouped of smoker and non smoker basically it is the mean of
# 2.222222 and 2.00000

In [234]:
tipper.pivot_table(index=['day','time'],columns='smoker',
                 values = ['size','tip'] , margins=True)

size                           tip                    
smoker             No       Yes       All        No       Yes       All
day  time                                                              
Fri  Dinner  2.000000  2.222222  2.166667  2.750000  3.003333  2.940000
     Lunch   3.000000  1.833333  2.000000  3.000000  2.280000  2.382857
Sat  Dinner  2.555556  2.476190  2.517241  3.102889  2.875476  2.993103
Sun  Dinner  2.929825  2.578947  2.842105  3.167895  3.516842  3.255132
Thur Dinner  2.000000       NaN  2.000000  3.000000       NaN  3.000000
     Lunch   2.500000  2.352941  2.459016  2.666364  3.030000  2.767705
All          2.668874  2.408602  2.569672  2.991854  3.008710  2.998279

In [235]:
# Now to use any other aggregate functions besides mean like count or len, pass them to the aggfunc keyword

# Note:- Count will ignore NA values but len will not

In [236]:
tipper.pivot_table(index=['day','time'],columns='smoker',
                 values = ['size','tip'] , margins=True, aggfunc = 'count')

size               tip           
smoker          No   Yes  All     No   Yes  All
day  time                                      
Fri  Dinner    3.0   9.0   12    3.0   9.0   12
     Lunch     1.0   6.0    7    1.0   6.0    7
Sat  Dinner   45.0  42.0   87   45.0  42.0   87
Sun  Dinner   57.0  19.0   76   57.0  19.0   76
Thur Dinner    1.0   NaN    1    1.0   NaN    1
     Lunch    44.0  17.0   61   44.0  17.0   61
All          151.0  93.0  244  151.0  93.0  244

In [237]:
tipper.pivot_table(index=['day','time'],columns='smoker',
                 values = ['size','tip'] , margins=True, aggfunc = len)

size               tip             
smoker          No   Yes  All     No   Yes    All
day  time                                        
Fri  Dinner    3.0   9.0   12    3.0   9.0   12.0
     Lunch     1.0   6.0    7    1.0   6.0    7.0
Sat  Dinner   45.0  42.0   87   45.0  42.0   87.0
Sun  Dinner   57.0  19.0   76   57.0  19.0   76.0
Thur Dinner    1.0   NaN    1    1.0   NaN    1.0
     Lunch    44.0  17.0   61   44.0  17.0   61.0
All          151.0  93.0  244  151.0  93.0  244.0

In [238]:

# Now you can fill the NA values with some default in the following way

In [239]:
tipper

,total_bill,tip,smoker,day,time,size,tip_prcnt
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [240]:
tipper.pivot_table(index=['time','size','smoker'],columns='day',
                 values = ['total_bill','tip'] ,fill_value='aaja raja')

tip                                  total_bill  \
day                       Fri        Sat        Sun       Thur        Fri   
time   size smoker                                                          
Dinner 1    No      aaja raja        1.0  aaja raja  aaja raja  aaja raja   
            Yes     aaja raja        1.0  aaja raja  aaja raja  aaja raja   
       2    No           2.75     2.5632   2.582222        3.0  19.233333   
            Yes        2.7875   2.476786      3.345  aaja raja   17.26125   
       3    No      aaja raja     3.4125   2.937273  aaja raja  aaja raja   
            Yes     aaja raja   4.568333      3.625  aaja raja  aaja raja   
       4    No      aaja raja        4.8     3.9625  aaja raja  aaja raja   
            Yes          4.73      3.335       5.09  aaja raja      40.17   
       5    No      aaja raja  aaja raja       5.07  aaja raja  aaja raja   
            Yes     aaja raja        3.0        2.0  aaja raja  aaja raja   
       6    No      aaja raja  aaja raja        5.0  aaja raja  aaja raja   
Lunch  1    No      aaja raja  aaja raja  aaja raja       1.83  aaja raja   
            Yes          1.92  aaja raja  aaja raja  aaja raja       8.58   
       2    No      aaja raja  aaja raja  aaja raja   2.322353  aaja raja   
            Yes         2.352  aaja raja  aaja raja   2.713846     13.072   
       3    No            3.0  aaja raja  aaja raja       1.77      15.98   
            Yes     aaja raja  aaja raja  aaja raja      3.615  aaja raja   
       4    No      aaja raja  aaja raja  aaja raja       4.03  aaja raja   
            Yes     aaja raja  aaja raja  aaja raja        4.5  aaja raja   
       5    No      aaja raja  aaja raja  aaja raja        5.0  aaja raja   
       6    No      aaja raja  aaja raja  aaja raja        5.3  aaja raja   

                                                     
day                       Sat        Sun       Thur  
time   size smoker                                   
Dinner 1    No           7.25  aaja raja  aaja raja  
            Yes          3.07  aaja raja  aaja raja  
       2    No        16.0264  15.642222      18.78  
            Yes     17.561071     21.875  aaja raja  
       3    No        21.9625  20.478182  aaja raja  
            Yes     32.603333     26.875  aaja raja  
       4    No      30.474286  26.410625  aaja raja  
            Yes     29.178333      28.91  aaja raja  
       5    No      aaja raja      25.27  aaja raja  
            Yes         28.15      30.46  aaja raja  
       6    No      aaja raja      48.17  aaja raja  
Lunch  1    No      aaja raja  aaja raja      10.07  
            Yes     aaja raja  aaja raja  aaja raja  
       2    No      aaja raja  aaja raja  14.156765  
            Yes     aaja raja  aaja raja  17.493846  
       3    No      aaja raja  aaja raja      20.73  
            Yes     aaja raja  aaja raja      17.59  
       4    No      aaja raja  aaja raja  28.703333  
            Yes     aaja raja  aaja raja      31.82  
       5    No      aaja raja  aaja raja      41.19  
       6    No      aaja raja  aaja raja  30.383333

### Cross-Tabulations:Crosstab

In [241]:
# A cross tab is a special case of pivot table that computes group frequencies

In [242]:
# Let's see an example

In [243]:
data = """ Sample Nationality Handedness
        1 USA Right-handed
        2 Japan left-handed
        3 USA Right-handed
        4 Japan Right-handed
        5 Japan left-handed
        6 Japan Right-handed
        7 USA Right-handed
        8 USA left-handed
        9 Japan Right-handed
        10 USA Right-handed
"""

data

' Sample Nationality Handedness\n        1 USA Right-handed\n        2 Japan left-handed\n        3 USA Right-handed\n        4 Japan Right-handed\n        5 Japan left-handed\n        6 Japan Right-handed\n        7 USA Right-handed\n        8 USA left-handed\n        9 Japan Right-handed\n        10 USA Right-handed\n'

In [244]:
data_as_DF = pd.read_table(StringIO(data),sep='\s+')

data_as_DF

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [245]:
# Suppose you want to summarize this data on basis of nationality and handedness

In [246]:
pd.crosstab(data_as_DF['Nationality'],data_as_DF['Handedness'],margins=True)

Handedness,Right-handed,left-handed,All
Nationality,,,
Japan,3,2,5
USA,4,1,5
All,7,3,10


In [247]:
# Now, note that the first two arguments can be a list or series or list of arrays

In [248]:
pd.crosstab([tipper['time'],tipper['day']],tipper['smoker'],margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

### 